In [5]:
import Utilitaires
num_classes=38



In [ ]:
import torch
import torch.utils.data

_,trainset=Utilitaires.txt_to_tensordataset('train2.txt',num_classes)
test,testset=Utilitaires.txt_to_tensordataset('test2.txt',num_classes)

trainloader=torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True,num_workers=0)
testloader=torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True,num_workers=0)
print(len(testloader.dataset))
print(len(trainloader.dataset))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [16]:
class VoxNet(torch.nn.Module):

    def __init__(self, num_classes, input_shape=(32, 32, 32)): 
        super(VoxNet, self).__init__()
        self.conv = torch.nn.Sequential(torch.nn.Conv3d(1,32,5,2),
                                       torch.nn.LeakyReLU(),
                                       torch.nn.Dropout(p=0.2),
                                       torch.nn.Conv3d(32, 32, 3),
                                       torch.nn.LeakyReLU(),
                                       torch.nn.MaxPool3d(2),
                                       torch.nn.Dropout(p=0.3)
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(32* 6 * 6 * 6, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.4),
            torch.nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x=x.view(x.size()[0],1,32,32,32)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x

    

In [17]:
import torch.nn as nn
criterion=nn.CrossEntropyLoss()
net = VoxNet(num_classes=num_classes)
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print("using cuda")
    net.cuda()
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
Utilitaires.train(net, optimizer, trainloader, testloader, criterion,  n_epoch = 10,
      train_acc_period = 10,
      test_acc_period = 1000)

using cuda


RuntimeError: CUDA error: out of memory

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
labels_anglais=['airplane',
 'bathtub',
 'bed',
 'bench',
 'bookshelf',
 'bottle',
 'bowl',
 'car',
 'chair',
 'cone',
 'cup',
 'curtain',
 'desk',
 'door',
 'dresser',
 'flower_pot',
 'glass_box',
 'guitar',
 'keyboard',
 'lamp',
 'laptop',
 'mantel',
 'monitor',
 'night_stand',
 'person',
 'piano',
 'plant',
 'radio',
 'range_hood',
 'sink',
 'sofa',
 'stairs',
 'stool',
 'table',
 'tent',
 'toilet',
 'tv_stand',
 'vase',
 'wardrobe',
 'xbox']
def testeur(net):#demonstration of algorithm, prediciton by VoxNet
  net.eval()
  l=[j for j in range(len(testloader.dataset))]
  shuffle(l)
  for i in range(16):
    j=l[i]
    volume=torch.Tensor.numpy(test[j])
    plot3DVoxel(volume[3:-3,3:-3,3:-3])
    #trace(volume[3:-3,3:-3,3:-3])
    temp=test[j].view(1,32,32,32)
    temp = temp.type(torch.cuda.FloatTensor)
    output=net(temp)
    p=torch.argmax(output, dim=1)
    print(labels_anglais[p])
  net.train()  

testeur(net)

In [11]:
Utilitaires.accuracy(net,testloader)

NameError: name 'testloader' is not defined